In [1]:
from scqbf.scqbf_instance import *
from scqbf.scqbf_solution import *
from scqbf.scqbf_evaluator import *
from scqbf.scqbf_ga import *
import os
#import glob
#import time
from concurrent.futures import ProcessPoolExecutor, as_completed
import pandas as pd
from exp_utils import run_single_experiment

In [2]:
instance_paths = [(f"instances/gen{i}/instance{j}.txt", i, j) for i in range(1, 4) for j in range(1, 6)]
instance_paths

[('instances/gen1/instance1.txt', 1, 1),
 ('instances/gen1/instance2.txt', 1, 2),
 ('instances/gen1/instance3.txt', 1, 3),
 ('instances/gen1/instance4.txt', 1, 4),
 ('instances/gen1/instance5.txt', 1, 5),
 ('instances/gen2/instance1.txt', 2, 1),
 ('instances/gen2/instance2.txt', 2, 2),
 ('instances/gen2/instance3.txt', 2, 3),
 ('instances/gen2/instance4.txt', 2, 4),
 ('instances/gen2/instance5.txt', 2, 5),
 ('instances/gen3/instance1.txt', 3, 1),
 ('instances/gen3/instance2.txt', 3, 2),
 ('instances/gen3/instance3.txt', 3, 3),
 ('instances/gen3/instance4.txt', 3, 4),
 ('instances/gen3/instance5.txt', 3, 5)]

In [3]:
def run_experiment(strategy: GAStrategy, pop_size: int = 100, mutation_rate: float = 1) -> pd.DataFrame:    
    """
    Run experiments in parallel using multiprocessing.
    """
    results = []
    
    experiment_args = [
        (instance_path, gen, inst, exp_num, strategy, pop_size, mutation_rate)
        for exp_num, (instance_path, gen, inst) in enumerate(sorted(instance_paths), start=1)
    ]

    max_workers = max(min(len(instance_paths), os.cpu_count() or 4) - 2, 1)

    try:
        with ProcessPoolExecutor(max_workers=max_workers) as executor:
            futures = [executor.submit(run_single_experiment, args) for args in experiment_args]
            
            for future in as_completed(futures):
                try:
                    result = future.result()
                    results.append(result)
                    print(f"Completed experiment {result['exp_num']}: Generation {result['gen']}, Instance {result['inst']}", flush=True)
                except Exception as e:
                    print(f"Experiment failed with error: {e}")
    except KeyboardInterrupt:
        print("Kernel interrupted. Shutting down worker processes...")
        executor.shutdown(wait=True, cancel_futures=True)
        raise  # Re-raise the KeyboardInterrupt to stop execution
        
    
    df = pd.DataFrame(results)
    df = df.sort_values(['n', 'gen']).reset_index(drop=True)
    
    for _, row in df.iterrows():
        print(f"{row['exp_num']}: {row['inst']}th instance of generation strategy {row['gen']}. Path: {row['instance_path']}")
        print(f"\tBest objective value: {row['best_objective']:.4f}")
        print(f"Selected elements: {row['selected_elements']}")
        print(f"\tCoverage: {row['coverage']:.2%}")
        print(f"\tTime taken (secs): {row['time_taken']:.4f} s")
        print(f"\tStop reason: {row['stop_reason']}")
        print()
    
    return df.drop(columns=['exp_num', 'instance_path', 'selected_elements', 'inst'])

# Experimentos

In [4]:
P1 = 100
P2 = 50

M1 = 1
M2 = 3

## Experimento 1: PADRÃO
Algoritmo Genético com tamanho de população P1 , taxa de mutação M1 e construção
aleatória da população.

In [5]:
run_experiment(pop_size=P1, mutation_rate=M1, strategy=GAStrategy(population_init='random', mutation_strategy='standard'))

Completed experiment 4: Generation 1, Instance 4
Completed experiment 8: Generation 2, Instance 3
Completed experiment 3: Generation 1, Instance 3
Completed experiment 6: Generation 2, Instance 1
Completed experiment 14: Generation 3, Instance 4
Completed experiment 15: Generation 3, Instance 5
Completed experiment 5: Generation 1, Instance 5
Completed experiment 12: Generation 3, Instance 2
Completed experiment 10: Generation 2, Instance 5
Completed experiment 2: Generation 1, Instance 2
Completed experiment 1: Generation 1, Instance 1
Completed experiment 9: Generation 2, Instance 4
Completed experiment 7: Generation 2, Instance 2
Completed experiment 11: Generation 3, Instance 1
Completed experiment 13: Generation 3, Instance 3
4: 4th instance of generation strategy 1. Path: instances/gen1/instance4.txt
	Best objective value: 261.0600
Selected elements: [0, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 16, 18, 19, 20, 23, 24]
	Coverage: 100.00%
	Time taken (secs): 4.0651 s
	Stop reason: pat

,gen,n,stop_reason,best_objective,coverage,time_taken
0,1,25,patience_exceeded,261.06,1.0,4.065121
1,2,25,patience_exceeded,210.16,1.0,4.604153
2,3,25,patience_exceeded,981.05,1.0,5.045109
3,3,25,patience_exceeded,766.71,1.0,4.758831
4,1,50,patience_exceeded,645.62,1.0,25.270666
5,2,50,patience_exceeded,575.87,1.0,30.974200
6,1,100,patience_exceeded,1525.48,1.0,237.627915
7,2,100,patience_exceeded,1515.43,1.0,347.853527
8,3,100,patience_exceeded,14887.43,1.0,265.629339
9,1,200,time_limit,4668.07,1.0,1800.240409


## Experimento 2: PADRÃO+POP
Algoritmo Genético PADRÃO mas com tamanho de população P2 .

In [6]:
run_experiment(pop_size=P2, mutation_rate=M1, strategy=GAStrategy(population_init='random', mutation_strategy='standard'))

Completed experiment 4: Generation 1, Instance 4
Completed experiment 8: Generation 2, Instance 3
Completed experiment 3: Generation 1, Instance 3
Completed experiment 6: Generation 2, Instance 1
Completed experiment 14: Generation 3, Instance 4
Completed experiment 15: Generation 3, Instance 5
Completed experiment 5: Generation 1, Instance 5
Completed experiment 10: Generation 2, Instance 5
Completed experiment 12: Generation 3, Instance 2
Completed experiment 2: Generation 1, Instance 2
Completed experiment 13: Generation 3, Instance 3
Completed experiment 1: Generation 1, Instance 1
Completed experiment 7: Generation 2, Instance 2
Completed experiment 9: Generation 2, Instance 4
Completed experiment 11: Generation 3, Instance 1
4: 4th instance of generation strategy 1. Path: instances/gen1/instance4.txt
	Best objective value: 261.0600
Selected elements: [0, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 16, 18, 19, 20, 23, 24]
	Coverage: 100.00%
	Time taken (secs): 1.8694 s
	Stop reason: pat

,gen,n,stop_reason,best_objective,coverage,time_taken
0,1,25,patience_exceeded,261.06,1.0,1.869442
1,2,25,patience_exceeded,210.16,1.0,2.323915
2,3,25,patience_exceeded,981.05,1.0,2.567234
3,3,25,patience_exceeded,766.71,1.0,2.262138
4,1,50,patience_exceeded,645.62,1.0,13.092042
5,2,50,patience_exceeded,575.87,1.0,15.387147
6,1,100,patience_exceeded,1522.90,1.0,90.625559
7,2,100,patience_exceeded,1529.54,1.0,114.224900
8,3,100,patience_exceeded,14887.43,1.0,121.755894
9,1,200,patience_exceeded,4659.04,1.0,626.807095


## Experimento 3: PADRÃO+MUT
Algoritmo Genético PADRÃO mas com taxa de mutação M2 .

In [7]:
run_experiment(pop_size=P1, mutation_rate=M2, strategy=GAStrategy(population_init='random', mutation_strategy='standard'))

Completed experiment 4: Generation 1, Instance 4
Completed experiment 8: Generation 2, Instance 3
Completed experiment 3: Generation 1, Instance 3
Completed experiment 6: Generation 2, Instance 1
Completed experiment 14: Generation 3, Instance 4
Completed experiment 15: Generation 3, Instance 5
Completed experiment 5: Generation 1, Instance 5
Completed experiment 10: Generation 2, Instance 5
Completed experiment 12: Generation 3, Instance 2
Completed experiment 2: Generation 1, Instance 2
Completed experiment 1: Generation 1, Instance 1
Completed experiment 7: Generation 2, Instance 2
Completed experiment 9: Generation 2, Instance 4
Completed experiment 11: Generation 3, Instance 1
Completed experiment 13: Generation 3, Instance 3
4: 4th instance of generation strategy 1. Path: instances/gen1/instance4.txt
	Best objective value: 261.0600
Selected elements: [0, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 16, 18, 19, 20, 23, 24]
	Coverage: 100.00%
	Time taken (secs): 7.3402 s
	Stop reason: pat

,gen,n,stop_reason,best_objective,coverage,time_taken
0,1,25,patience_exceeded,261.06,1.0,7.340203
1,2,25,patience_exceeded,206.89,1.0,7.816857
2,3,25,patience_exceeded,945.84,1.0,6.438518
3,3,25,patience_exceeded,766.71,1.0,6.947424
4,1,50,patience_exceeded,628.15,1.0,45.321937
5,2,50,patience_exceeded,575.33,1.0,96.877382
6,1,100,patience_exceeded,1480.19,1.0,455.039137
7,2,100,patience_exceeded,1530.02,1.0,1040.951661
8,3,100,patience_exceeded,14887.43,1.0,1211.526734
9,1,200,time_limit,4459.87,1.0,1800.038427


## Experimento 4: PADRÃO+EVOL1
Algoritmo Genético PADRÃO mas com estratégia evolutiva alternativa 1 (Latin Hypercube)

In [8]:
run_experiment(pop_size=P1, mutation_rate=M1, strategy=GAStrategy(population_init='latin_hypercube', mutation_strategy='standard'))

Completed experiment 4: Generation 1, Instance 4
Completed experiment 8: Generation 2, Instance 3
Completed experiment 3: Generation 1, Instance 3
Completed experiment 6: Generation 2, Instance 1
Completed experiment 14: Generation 3, Instance 4
Completed experiment 15: Generation 3, Instance 5
Completed experiment 5: Generation 1, Instance 5
Completed experiment 10: Generation 2, Instance 5
Completed experiment 12: Generation 3, Instance 2
Completed experiment 2: Generation 1, Instance 2
Completed experiment 1: Generation 1, Instance 1
Completed experiment 7: Generation 2, Instance 2
Completed experiment 9: Generation 2, Instance 4
Completed experiment 11: Generation 3, Instance 1
Completed experiment 13: Generation 3, Instance 3
4: 4th instance of generation strategy 1. Path: instances/gen1/instance4.txt
	Best objective value: 261.0600
Selected elements: [0, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 16, 18, 19, 20, 23, 24]
	Coverage: 100.00%
	Time taken (secs): 3.4607 s
	Stop reason: pat

,gen,n,stop_reason,best_objective,coverage,time_taken
0,1,25,patience_exceeded,261.06,1.0,3.460694
1,2,25,patience_exceeded,210.16,1.0,4.214345
2,3,25,patience_exceeded,981.05,1.0,4.935599
3,3,25,patience_exceeded,766.71,1.0,4.701787
4,1,50,patience_exceeded,645.62,1.0,28.473275
5,2,50,patience_exceeded,575.87,1.0,28.535522
6,1,100,patience_exceeded,1508.25,1.0,165.387435
7,2,100,patience_exceeded,1550.49,1.0,237.834497
8,3,100,patience_exceeded,14887.43,1.0,257.803497
9,1,200,patience_exceeded,4614.92,1.0,1244.885900


## Experimento 5: PADRÃO+EVOL2
Algoritmo Genético PADRÃO mas com estratégia evolutiva alternativa 2 (adaptive mutation)

In [9]:
run_experiment(pop_size=P1, mutation_rate=M1, strategy=GAStrategy(population_init='random', mutation_strategy='adaptive'))

Completed experiment 4: Generation 1, Instance 4
Completed experiment 8: Generation 2, Instance 3
Completed experiment 3: Generation 1, Instance 3
Completed experiment 6: Generation 2, Instance 1
Completed experiment 14: Generation 3, Instance 4
Completed experiment 15: Generation 3, Instance 5
Completed experiment 12: Generation 3, Instance 2
Completed experiment 5: Generation 1, Instance 5
Completed experiment 10: Generation 2, Instance 5
Completed experiment 1: Generation 1, Instance 1
Completed experiment 2: Generation 1, Instance 2
Completed experiment 7: Generation 2, Instance 2
Completed experiment 9: Generation 2, Instance 4
Completed experiment 11: Generation 3, Instance 1
Completed experiment 13: Generation 3, Instance 3
4: 4th instance of generation strategy 1. Path: instances/gen1/instance4.txt
	Best objective value: 261.0600
Selected elements: [0, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 16, 18, 19, 20, 23, 24]
	Coverage: 100.00%
	Time taken (secs): 3.4754 s
	Stop reason: pat

,gen,n,stop_reason,best_objective,coverage,time_taken
0,1,25,patience_exceeded,261.06,1.0,3.475364
1,2,25,patience_exceeded,210.16,1.0,4.207716
2,3,25,patience_exceeded,981.05,1.0,5.162829
3,3,25,patience_exceeded,766.71,1.0,4.964608
4,1,50,patience_exceeded,645.62,1.0,26.813301
5,2,50,patience_exceeded,575.87,1.0,36.725098
6,1,100,patience_exceeded,1489.16,1.0,370.170277
7,2,100,patience_exceeded,1512.17,1.0,524.802653
8,3,100,patience_exceeded,14887.43,1.0,348.146095
9,1,200,time_limit,4349.99,1.0,1800.190899


# Experimento 6
Estratégia evolutiva 2 com população P2

In [13]:
run_experiment(pop_size=P2, mutation_rate=M1, strategy=GAStrategy(population_init='latin_hypercube', mutation_strategy='standard'))

Completed experiment 4: Generation 1, Instance 4
Completed experiment 8: Generation 2, Instance 3
Completed experiment 3: Generation 1, Instance 3
Completed experiment 6: Generation 2, Instance 1
Completed experiment 14: Generation 3, Instance 4
Completed experiment 15: Generation 3, Instance 5
Completed experiment 5: Generation 1, Instance 5
Completed experiment 10: Generation 2, Instance 5
Completed experiment 12: Generation 3, Instance 2
Completed experiment 1: Generation 1, Instance 1
Completed experiment 2: Generation 1, Instance 2
Completed experiment 13: Generation 3, Instance 3
Completed experiment 9: Generation 2, Instance 4
Completed experiment 7: Generation 2, Instance 2
Completed experiment 11: Generation 3, Instance 1
4: 4th instance of generation strategy 1. Path: instances/gen1/instance4.txt
	Best objective value: 261.0600
Selected elements: [0, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 16, 18, 19, 20, 23, 24]
	Coverage: 100.00%
	Time taken (secs): 2.3609 s
	Stop reason: pat

,gen,n,stop_reason,best_objective,time_taken
0,1,25,patience_exceeded,261.06,2.360867
1,2,25,patience_exceeded,210.16,2.615304
2,3,25,patience_exceeded,981.05,2.901450
3,3,25,patience_exceeded,766.71,2.417926
4,1,50,patience_exceeded,645.62,12.448971
5,2,50,patience_exceeded,575.87,15.671818
6,1,100,patience_exceeded,1501.12,93.732521
7,2,100,patience_exceeded,1471.07,113.927180
8,3,100,patience_exceeded,14887.43,127.576004
9,1,200,patience_exceeded,4467.85,538.195891
